<a href="https://colab.research.google.com/github/trungpro5398/Cat_And_Dog_Classification/blob/main/Cat_Dog_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install torch torchvision transformers datasets transformers[torch] accel typing_extensions kaleido cohere openai tiktoken gradio


In [16]:
pip install transformers[torch] accelerate -U


In [17]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load the dataset from Hugging Face Datasets Hub
dataset = load_dataset("Bingsu/Cat_and_Dog")

# Split dataset into train and validation
train_size = int(0.8 * len(dataset['train']))
val_size = len(dataset['train']) - train_size

# Create data loaders with the correct labels and pixel values
train_dataset = dataset['train'].map(lambda example: {'pixel_values': transform(example['image']), 'labels': example['labels']})
val_dataset = dataset['train'].map(lambda example: {'pixel_values': transform(example['image']), 'labels': example['labels']}).train_test_split(test_size=val_size)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset['train'], batch_size=32)

# Load the ViT model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # You can customize the data collator if needed
    train_dataset=train_dataset,
    eval_dataset=val_dataset['train']
)

# Train the model
trainer.train()


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.753100
20,0.741100
30,0.733100
40,0.709000
50,0.675400
60,0.641500
70,0.599200
80,0.533000
90,0.458200
100,0.372400


TrainOutput(global_step=750, training_loss=0.11696217236419519, metrics={'train_runtime': 2436.5362, 'train_samples_per_second': 9.85, 'train_steps_per_second': 0.308, 'total_flos': 1.859807750750208e+18, 'train_loss': 0.11696217236419519, 'epoch': 3.0})

In [18]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.00484069250524044, 'eval_runtime': 622.2439, 'eval_samples_per_second': 10.285, 'eval_steps_per_second': 0.321, 'epoch': 3.0}


In [19]:
# Define the directory where you want to save the model
model_save_directory = "/content/my_model"

# Save the model and the tokenizer/feature extractor
model.save_pretrained(model_save_directory)
feature_extractor.save_pretrained(model_save_directory)

['/content/my_model/preprocessor_config.json']

In [20]:
import gradio as gr

model_save_directory = "/content/my_model"

# Function to load the model and feature extractor
def load_model_and_feature_extractor():
    model = ViTForImageClassification.from_pretrained(model_save_directory)
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_save_directory)
    return model, feature_extractor

# Load the model and feature extractor
model, feature_extractor = load_model_and_feature_extractor()
# This is your prediction function that Gradio will use
def predict_image_with_gradio(image):
    # Transform the image to the format expected by the model
    image = feature_extractor(images=image, return_tensors="pt").to(model.device)

    # Get predictions
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(**image)

    # Convert output logits to softmax probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)

    # Get the top prediction
    top_prob, top_catid = torch.topk(probabilities, 1)
    predicted_label = model.config.id2label[top_catid.item()]
    probability = top_prob.item()

    # Construct the results dictionary with Python's if-else expressions
    return {
        "Cat": probability if predicted_label == 'LABEL_0' else (1 - probability),
        "Dog": probability if predicted_label == 'LABEL_1' else (1 - probability)
    }


# Define the input and output components for the Gradio interface
input_component = "image"
output_component = gr.Label(num_top_classes=2)

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_image_with_gradio,
    inputs=input_component,
    outputs=output_component,
    title="Cat and Dog Image Classifier",
    description="Upload an image and the model will predict whether it's a cat or dog.",
)

iface.launch()

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://62da12948fa8197e12.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [21]:
from google.colab import drive
drive.mount('/content/drive')
model_save_directory = '/content/drive/My Drive/my_model_directory'
import os
os.makedirs(model_save_directory, exist_ok=True)
model.save_pretrained(model_save_directory)
feature_extractor.save_pretrained(model_save_directory)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['/content/drive/My Drive/my_model_directory/preprocessor_config.json']